In [1]:
import pandas as pd
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *

spark = (SparkSession.builder
    .master('local[*]')
    .config("spark.driver.memory", "15g")
    .appName('spark')
    .getOrCreate())

In [2]:
tim = (spark.read.csv('/Users/irene/Desktop/DAILYMED_DISEASE_ChEMBL_ML_2.tsv', sep='\t', header=True)
    .select('chembl_id', 'efo_id', 'ref_url', 'method_by')
    .withColumn('efo_id', F.regexp_replace(F.col('efo_id'), ':', '_'))
    .withColumn('dailymed_id', F.element_at(F.split(F.col('ref_url'), '='), -1)).drop('ref_url').distinct()

    .filter(F.col('method_by') == 'ml').drop('method_by')
    )
tim.first()

Row(chembl_id='CHEMBL1751', efo_id='EFO_0003144', dailymed_id='58678fd6-afe3-4a4c-a37f-356444194cd6')

In [3]:
print(tim.distinct().count())
print(tim.select('dailymed_id').distinct().count())
tim.select('chembl_id', 'efo_id').distinct().count()

8469
3946


2832

In [4]:
'''
tim_expanded = (

    tim.join(disease_idx, disease_idx['id'] == tim['efo_id'], how='left')
        .withColumn('efo_id', F.coalesce('ids', F.array(F.col('efo_id'))))
        .withColumn('efo_id', F.explode('efo_id'))
        .distinct()
        .persist()

)
'''
;

''

In [5]:
# Current manual indications

ind = spark.read.parquet('/Users/irene/Documents/dev/pyspark/21.08/indication')

In [6]:
# Extract dailymed refs

dailymed = (
    ind.withColumn('indication', F.explode('indications'))
        .withColumn('reference', F.explode('indication.references'))
        .filter(F.col('reference.source') == 'DailyMed')
        .withColumn('efo_id', F.col('indication.disease'))
        .withColumn('dailymed_id', F.explode('reference.ids'))
        .drop('approvedIndications', 'indications', 'references', 'reference', 'indication', 'indicationCount')
        .distinct()
)

dailymed.first()

Row(id='CHEMBL480', efo_id='EFO_0003948', dailymed_id='da982c79-4f29-467c-b237-1f13c2051c59')

In [7]:
print(dailymed.distinct().count())
print(dailymed.select('dailymed_id').distinct().count())
dailymed.select('id', 'efo_id').distinct().count()

24720
9510


2552

In [8]:
## Compare datasets

In [9]:
print(dailymed.join(tim, on='efo_id', how='left_anti').distinct().count())
print(tim.join(dailymed, on='efo_id', how='left_anti').select('efo_id').distinct().count())
# 2504 EFOs are lost
# 337 are gained

2504
337


In [10]:
print(dailymed.join(tim, dailymed['id'] == tim['chembl_id'], how='left_anti').select('id').distinct().count())
tim.join(dailymed, dailymed['id'] == tim['chembl_id'], how='left_anti').select('chembl_id').distinct().count()

# 516 chembl ids are lost
# 548 are gained

516


548

In [11]:
dailymed.join(tim, dailymed['id'] == tim['chembl_id'], how='left_anti').select('id', 'dailymed_id').first()

Row(id='CHEMBL480', dailymed_id='da982c79-4f29-467c-b237-1f13c2051c59')

In [12]:
dailymed.join(tim, on='dailymed_id', how='left_anti').select('dailymed_id').distinct().show(truncate=False)

+------------------------------------+
|dailymed_id                         |
+------------------------------------+
|f70e7022-0c44-48f2-9f7e-606e7ac730aa|
|de965605-268a-4479-86f7-84de949cf36f|
|f625d3e3-c175-4d9a-99ea-dab410b262f1|
|9b93035b-34d6-4084-ba73-5dea72f9d124|
|759d754e-48eb-4e37-a28a-9b79ac20416b|
|331733b8-9da7-4901-ad1f-b7c088730f8c|
|c3335a21-20c9-46a5-5d9f-12227d67fd11|
|f2bd24a1-01cd-461b-a330-0456a0e5ee58|
|c8c7e092-0eb4-4945-934d-0b4cf664ad23|
|f9d9d64a-3a3c-44ad-b40d-02789f29eb04|
|e054ec39-06e2-43ec-88cc-28095f76c6f5|
|80adf27d-60fb-4394-8881-e75d3acb2774|
|b1397a52-d99f-44f0-8649-8ceb8e7b9b85|
|001dd372-2692-4d6f-a71c-95e7056349a9|
|93ee7bb4-4be0-45a4-8f5b-6b17c9f8cfdf|
|b17f29ca-517e-4d89-84f1-024d98fc5d77|
|d224143d-2199-4969-a64f-dc2e6b865ec4|
|52321fe5-c001-4e2d-8149-3197ddc13764|
|a03f40f5-068e-41f2-826d-6ed627dd8776|
|6d0c2df4-e9db-4622-95e8-33aa8a08f593|
+------------------------------------+
only showing top 20 rows



In [13]:
print(dailymed.join(tim, on='dailymed_id', how='left_anti').select('dailymed_id').distinct().count())
tim.join(dailymed, on='dailymed_id', how='left_anti').select('dailymed_id').distinct().count()

# 8710 references are lost
# 3146 are gained

8710


3146

In [14]:
print(dailymed.withColumnRenamed('id', 'chembl_id').join(tim, ['chembl_id', 'efo_id'], how='left_anti').select('chembl_id', 'efo_id').distinct().count())
tim.join(dailymed.withColumnRenamed('id', 'chembl_id'), ['chembl_id', 'efo_id'], how='left_anti').select('chembl_id', 'efo_id').distinct().count()

# 1848 drug/indication pairs are lost
# 2128 are gained

1848


2128

In [15]:
print(dailymed.select('id').distinct().count())
print(tim.select('chembl_id').distinct().count())

1170
1202


In [16]:
'''
# are the lost references the source of the lost chembl ids? no. There are only 42 diff chembl ids in lost_refs.

lost_refs = dailymed.join(tim, on='dailymed_id', how='left_anti').select('id').distinct()

lost_refs.select('id').distinct().join(tim.select('chembl_id').distinct(), tim['chembl_id'] == lost_refs['id'], how='left_anti').distinct().count()
'''
;

''

In [17]:
# Taking the manual curation as the baseline, I want to see if the annotation is related

efo_ancestors = (
    spark.read.parquet('/Users/irene/Documents/dev/pyspark/21.09.5/diseases')
        .select(F.array(F.col('id')).alias('id'), 'ancestors')
        .withColumn('expanded_efo_id', F.array_union(F.col('id'), F.col('ancestors')))
        .withColumn('efo_id', F.explode('id'))
        .select('efo_id', 'expanded_efo_id')
)

tim_w_ancestors = tim.join(efo_ancestors, on='efo_id', how='left')
tim_w_ancestors.show(5)

+-----------+-------------+--------------------+--------------------+
|     efo_id|    chembl_id|         dailymed_id|     expanded_efo_id|
+-----------+-------------+--------------------+--------------------+
|EFO_0003144|   CHEMBL1751|58678fd6-afe3-4a4...|[EFO_0003144, MON...|
|EFO_0006505|CHEMBL1200735|7d74dfa6-0468-43a...|[EFO_0006505, MON...|
|EFO_0000676|    CHEMBL424|af7060a9-986c-500...|[EFO_0000676, OTA...|
| HP_0002148|CHEMBL1200925|475f5cd7-45bd-412...|[HP_0002148, EFO_...|
| HP_0000989|CHEMBL1201089|6f68cbbb-6269-4bb...|[HP_0000989, HP_0...|
+-----------+-------------+--------------------+--------------------+
only showing top 5 rows



In [18]:
# Grouping by drug and reference. Is the mapping related? -->
# --> Is the curated mapping present in any of the ancestors in Tim's annotation

enrichment = (
    dailymed.withColumnRenamed('id', 'chembl_id')
        .join(
        tim_w_ancestors.withColumnRenamed('efo_id', 'new_efo_id'),
        on = ['dailymed_id', 'chembl_id'],
        how='left')
        .distinct()
        .withColumn('sameMapping', F.when(F.col('efo_id') == F.col('new_efo_id'), True).otherwise(False))
        .withColumn('isRelated', F.array_contains(F.col('expanded_efo_id'), F.col('efo_id')))
        .withColumn('isRelated', F.when(F.col('isRelated').isNull(), False).otherwise(F.col('isRelated')))
)

enrichment.show()

+--------------------+-------------+-----------+-----------+--------------------+-----------+---------+
|         dailymed_id|    chembl_id|     efo_id| new_efo_id|     expanded_efo_id|sameMapping|isRelated|
+--------------------+-------------+-----------+-----------+--------------------+-----------+---------+
|000dc81d-ab91-450...|   CHEMBL1069|EFO_0003144|EFO_0003144|[EFO_0003144, MON...|       true|     true|
|000dc81d-ab91-450...|   CHEMBL1069|EFO_0000373|EFO_0003144|[EFO_0003144, MON...|      false|    false|
|0038f081-a867-43e...|CHEMBL1200636|EFO_1000632|       null|                null|      false|    false|
|0038f081-a867-43e...|CHEMBL1200636| HP_0001257|       null|                null|      false|    false|
|0038f081-a867-43e...|CHEMBL1200636|EFO_0009488|       null|                null|      false|    false|
|0038f081-a867-43e...|CHEMBL1200636| HP_0003326|       null|                null|      false|    false|
|025876b6-f031-43e...|   CHEMBL1163|EFO_0000764|       null|    

In [19]:
enrichment.select('dailymed_id', 'chembl_id', 'efo_id', 'isRelated').distinct().groupBy('isRelated').count().show()

+---------+-----+
|isRelated|count|
+---------+-----+
|     true|  766|
|    false|24358|
+---------+-----+



In [20]:
# It is not so trivial to check the enrichment of only the current indications.
# The explosion is so big that the counts are not informative enough
# I'll break it into parts

sameMapping = (
    dailymed.withColumnRenamed('id', 'chembl_id')
        .join(
        tim_w_ancestors,
        on = ['dailymed_id', 'chembl_id', 'efo_id'],
        how='inner')
        .distinct()
)

diffMapping = (
    dailymed.withColumnRenamed('id', 'chembl_id')
        .join(
        tim_w_ancestors,
        on = ['dailymed_id', 'chembl_id', 'efo_id'],
        how='left_anti')
        .distinct()
)

print('Annotation with different mapping', diffMapping.count())
print('Annotation with same mapping', sameMapping.count())

Annotation with different mapping 24031
Annotation with same mapping 689


In [21]:
# A record equals to a reference/drug/indication relationship
dailymed.select('dailymed_id', F.col('id').alias('chembl_id'), 'efo_id').distinct().join(tim.select('dailymed_id', 'chembl_id', 'efo_id').distinct(), on=['dailymed_id', 'chembl_id', 'efo_id'], how='left_anti').distinct().count()

24031

In [22]:
sameMapping

DataFrame[dailymed_id: string, chembl_id: string, efo_id: string, expanded_efo_id: array<string>]

In [23]:
diffMapping.show()

+--------------------+-------------+-----------+
|         dailymed_id|    chembl_id|     efo_id|
+--------------------+-------------+-----------+
|00f67e1a-6999-429...|   CHEMBL1237|EFO_0000537|
|01139e81-a4fa-4e1...|   CHEMBL1491|EFO_1000013|
|03cdc2f8-78d1-11d...|    CHEMBL485|EFO_0003843|
|0f81f505-a962-414...|     CHEMBL38|EFO_0000224|
|10a64abf-9704-4ab...|    CHEMBL894|EFO_0003761|
|10a7ba02-42d6-4ca...| CHEMBL393220|EFO_0003913|
|112fdec3-882b-14d...|    CHEMBL154|EFO_0002506|
|15cbb661-b92e-4fa...|    CHEMBL716|EFO_0000289|
|1bc4c6c8-7e75-494...|    CHEMBL141|EFO_0000764|
|1c20ce12-06d3-406...|   CHEMBL1694|EFO_0000537|
|1d99ad89-82b1-484...|    CHEMBL154|EFO_0002609|
|1e958146-6457-44e...|CHEMBL1201032|EFO_0003821|
|24a02cd6-667c-4e0...|     CHEMBL85|EFO_0003758|
|24cde2c2-dac7-4cd...|   CHEMBL1431|EFO_0001360|
|2596b87c-de70-4bf...| CHEMBL389621| MP_0001845|
|2676f761-4fe5-40a...|    CHEMBL190|EFO_0000270|
|27f9a2d4-9783-4a5...|    CHEMBL870|EFO_0003882|
|2892f3c2-bff6-441..

In [25]:
(
    diffMapping
        .join(
        tim_w_ancestors.withColumnRenamed('efo_id', 'new_efo_id'),
        on = ['dailymed_id', 'chembl_id'],
        how='left')
        .distinct()
        .filter(F.col('new_efo_id').isNotNull())
        .withColumn('isRelated', F.array_contains(F.col('expanded_efo_id'), F.col('efo_id')))
        .withColumn('isRelated', F.when(F.col('isRelated').isNull(), False).otherwise(F.col('isRelated')))
        #.select('dailymed_id', 'chembl_id', 'efo_id').distinct()
        .show()
)

+--------------------+-------------+-------------+-------------+--------------------+---------+
|         dailymed_id|    chembl_id|       efo_id|   new_efo_id|     expanded_efo_id|isRelated|
+--------------------+-------------+-------------+-------------+--------------------+---------+
|144067b2-0532-476...|    CHEMBL885|  EFO_0000764|  EFO_0000180|[EFO_0000180, EFO...|     true|
|80e736d3-2017-4d6...|   CHEMBL1430| Orphanet_905|MONDO_0003937|[MONDO_0003937, O...|    false|
|89595dbd-2722-41f...|     CHEMBL85|  EFO_0003015|  EFO_0003758|[EFO_0003758, MON...|    false|
|00088ecd-e45d-408...|CHEMBL3184512|  EFO_0008588|  EFO_0003102|[EFO_0003102, EFO...|    false|
|457a026c-ebce-470...|    CHEMBL730|  EFO_0000537|  EFO_0003144|[EFO_0003144, MON...|    false|
|deb4a13c-855b-437...|CHEMBL1201129|  EFO_0003811|  EFO_0000198|[EFO_0000198, OTA...|    false|
|4ea10b58-88d0-4d7...|    CHEMBL435|  EFO_0000537|  EFO_0000373|[EFO_0000373, MON...|    false|
|0fd36cb9-c4f6-416...|CHEMBL1201566|  EF

In [26]:
# Key thing here: when Tim annotates the same drug in a reference, most likely the EFO ID is the same.

# Question: What is the reason for annotating a different drug?

In [27]:
tim.joi

558

In [28]:
tim_w_ancestors.select('chembl_id', 'dailymed_id', 'efo_id').distinct().join(dailymed.withColumnRenamed('id', 'chembl_id').select('chembl_id', 'dailymed_id', 'efo_id').distinct(), on=['chembl_id', 'dailymed_id', 'efo_id'], how='left_anti').count()

7780

In [29]:
dailymed.filter(F.col('dailymed_id').contains('fe3e2db6-1aee-4767-bc1f')).show()

+-------------+-----------+--------------------+
|           id|     efo_id|         dailymed_id|
+-------------+-----------+--------------------+
|CHEMBL1200328|EFO_0003761|fe3e2db6-1aee-476...|
|   CHEMBL1175|EFO_0003761|fe3e2db6-1aee-476...|
+-------------+-----------+--------------------+



In [30]:
tim_w_ancestors.filter(F.col('dailymed_id').contains('fe3e2db6-1aee-4767-bc1f')).show()

+------+---------+-----------+---------------+
|efo_id|chembl_id|dailymed_id|expanded_efo_id|
+------+---------+-----------+---------------+
+------+---------+-----------+---------------+



In [31]:
# Finding: it is mainly because Tim only annotates one drug per reference, whereas the manual curation gets more chembl ids.

In [32]:
dailymed.select('id').distinct().count()

1170